In [1]:
import torch
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1' # 下面老是报错 shape 不一致

ckpt_path = 'ckpt/checkpoint.pt'
checkpoint = torch.load(ckpt_path, map_location=torch.device('cpu'))

epoch =  checkpoint['epoch']
best_val_mae = checkpoint['best_val_mae']

In [2]:
checkpoint

{'epoch': 91,
 'model_state_dict': OrderedDict([('module.x_embedding.weight',
               tensor([[ 0.0338, -0.0916,  0.0289,  ...,  0.0811,  0.1232,  0.0990],
                       [-0.0173,  0.0262, -0.0698,  ...,  0.0548, -0.1404, -0.0560],
                       [ 0.0510, -0.0243, -0.0095,  ..., -0.0709, -0.0384, -0.0143],
                       ...,
                       [-0.0361, -0.0306, -0.0692,  ...,  0.0785,  0.0315,  0.0289],
                       [-0.0094, -0.0347,  0.1224,  ..., -0.0886,  0.1231,  0.0480],
                       [ 0.0236, -0.0441,  0.0161,  ..., -0.1263,  0.0027, -0.0104]])),
              ('module.x_seg_embedding.weight',
               tensor([[ 0.0997, -0.0758,  0.1512,  ..., -0.0007,  0.2159, -0.0087],
                       [-0.0616, -0.0868, -0.0347,  ...,  0.0782, -0.0442,  0.0372],
                       [ 0.0536,  0.0516,  0.0368,  ...,  0.0857,  0.0968,  0.1084]])),
              ('module.edge_embedding.weight',
               tensor([[ 0.0

In [62]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem

sdf_data = Chem.SDMolSupplier('/home/gxzy/ThreeD/data/pcqm4m-v2/raw/pcqm4m-v2-train.sdf',
                                sanitize=True, removeHs=True, strictParsing=True)

mol1 = sdf_data[0]
mol2 = sdf_data[1]

position1 = torch.tensor(mol1.GetConformer().GetPositions(), dtype=torch.float)
position2 = torch.tensor(mol1.GetConformer().GetPositions(), dtype=torch.float)

from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

edge_index1 = torch.tensor([
   [0, 1, 1, 2],
   [1, 0, 2, 1],
])

edge_index2 = torch.tensor([
     [0, 0, 1, 1],
     [0, 1, 1, 2],
 ])

# class MyData(Data):
#     def __cat_dim__(self, key, value, *args, **kwargs):
#         if key == 'pos':
#             return 0
#         else:
#             return super().__cat_dim__(key, value, *args, **kwargs)

data1 = Data(edge_index=edge_index1, pos=position1)
data1.num_nodes = 18
data2 = Data(edge_index=edge_index2, pos=position2)
data2.num_nodes = 18


In [63]:
loader = DataLoader([data1, data2], batch_size=2)
batchx = next(iter(loader))
batchx

DataBatch(edge_index=[2, 8], pos=[36, 3], num_nodes=36, batch=[36], ptr=[3])